In [1]:
import torch
from torch import nn
import numpy as np
from torchsummary import summary
import compress_entropy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary(compress_entropy.Compress().to(device), (4, 768, 512))  ###  IDK why, but I have to switch the dims in AvgPool on Loss() to get this to work :/ But that breaks the stuff bellow

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 768, 512]             288
               ELU-2          [-1, 8, 768, 512]               0
            Conv2d-3          [-1, 8, 768, 512]             576
               ELU-4          [-1, 8, 768, 512]               0
            Conv2d-5          [-1, 8, 768, 512]             576
               ELU-6          [-1, 8, 768, 512]               0
            Conv2d-7          [-1, 8, 768, 512]              64
         MaxPool2d-8          [-1, 8, 768, 512]               0
               ELU-9          [-1, 8, 768, 512]               0
           Conv2d-10         [-1, 16, 768, 512]              64
             Stem-11         [-1, 16, 768, 512]               0
           Conv2d-12          [-1, 8, 768, 512]             128
              ELU-13          [-1, 8, 768, 512]               0
           Conv2d-14          [-1, 8, 7